In [1]:
pip install datascience

     |████████████████████████████████| 727 kB 616 kB/s eta 0:00:01
     |████████████████████████████████| 95 kB 814 kB/s eta 0:00:01
  Created wheel for datascience: filename=datascience-0.17.5-py3-none-any.whl size=732546 sha256=69e03ea46535d611b20ee32a1d7f1d9ff9ef3ebb7e46c8641cdd2fe85c7d9027
  Stored in directory: /home/nieece/.cache/pip/wheels/74/0a/88/e52ce8c2533663326372f4be6eba170267fb70dc6252234186
Successfully built datascience
Note: you may need to restart the kernel to use updated packages.


In [2]:
!pip install datascience


In [3]:
import pandas as pd
from scipy.io.arff import loadarff
from datascience import *
import numpy as np

In [6]:
raw_data=loadarff('/home/nieece/Downloads/weather.numeric.arff')
df_data=pd.DataFrame(raw_data[0])

In [7]:
df_data

,outlook,temperature,humidity,windy,play
0,b'sunny',85.0,85.0,b'FALSE',b'no'
1,b'sunny',80.0,90.0,b'TRUE',b'no'
2,b'overcast',83.0,86.0,b'FALSE',b'yes'
3,b'rainy',70.0,96.0,b'FALSE',b'yes'
4,b'rainy',68.0,80.0,b'FALSE',b'yes'
5,b'rainy',65.0,70.0,b'TRUE',b'no'
6,b'overcast',64.0,65.0,b'TRUE',b'yes'
7,b'sunny',72.0,95.0,b'FALSE',b'no'
8,b'sunny',69.0,70.0,b'FALSE',b'yes'
9,b'rainy',75.0,80.0,b'FALSE',b'yes'


In [8]:
from sklearn.preprocessing import LabelEncoder

LE=LabelEncoder()
df_data['outlook_n']=LE.fit_transform(df_data['outlook'])
df_data['windy_n']=LE.fit_transform(df_data['windy'])
df_data['play_n']=LE.fit_transform(df_data['play'])
print(df_data)

        outlook  temperature  humidity     windy    play  outlook_n  windy_n  \
0      b'sunny'         85.0      85.0  b'FALSE'   b'no'          2        0   
1      b'sunny'         80.0      90.0   b'TRUE'   b'no'          2        1   
2   b'overcast'         83.0      86.0  b'FALSE'  b'yes'          0        0   
3      b'rainy'         70.0      96.0  b'FALSE'  b'yes'          1        0   
4      b'rainy'         68.0      80.0  b'FALSE'  b'yes'          1        0   
5      b'rainy'         65.0      70.0   b'TRUE'   b'no'          1        1   
6   b'overcast'         64.0      65.0   b'TRUE'  b'yes'          0        1   
7      b'sunny'         72.0      95.0  b'FALSE'   b'no'          2        0   
8      b'sunny'         69.0      70.0  b'FALSE'  b'yes'          2        0   
9      b'rainy'         75.0      80.0  b'FALSE'  b'yes'          1        0   
10     b'sunny'         75.0      70.0   b'TRUE'  b'yes'          2        1   
11  b'overcast'         72.0      90.0  

In [9]:
df_update=df_data.drop(columns=['outlook','windy','play'])
print(df_update)

    temperature  humidity  outlook_n  windy_n  play_n
0          85.0      85.0          2        0       0
1          80.0      90.0          2        1       0
2          83.0      86.0          0        0       1
3          70.0      96.0          1        0       1
4          68.0      80.0          1        0       1
5          65.0      70.0          1        1       0
6          64.0      65.0          0        1       1
7          72.0      95.0          2        0       0
8          69.0      70.0          2        0       1
9          75.0      80.0          1        0       1
10         75.0      70.0          2        1       1
11         72.0      90.0          0        1       1
12         81.0      75.0          0        0       1
13         71.0      91.0          1        1       0


In [40]:
def distance_between_points(pt1,pt2):
    """return the distance between two points"""
    return (np.sqrt(sum((pt1-pt2)**2)))

def distance_between_rows(row1,row2):
    """return the distance between two numerical rows of a table"""
    return distance_between_points(np.array(row1,dtype=float),np.array(row2,dtype=float))

In [41]:
def distances(training,example):
    """Compute a table with the dataset and distances to the example for each row in the training set"""
    dists=[]
    attributes=training.drop('play_n')
    for row in attributes.rows:
        dist=distance_between_rows(row,example)
        dists.append(dist)
    return training.with_column('Distance',dists)

In [42]:
def closest(training,example,k):
    """1.make a separate table for distance between example row amd all other rows
       2.sort in ascending
       3.take first 5(closest distance between example row and all other rows)
       return a table of k closest neighbors to example"""
    return distances(training,example).sort('Distance').take(np.arange(k))

In [43]:
#consider 1 row
Dataset=Table.from_df(df_update)

In [44]:
#test between two rows
attributes=Dataset.drop('play_n')
attributes.show(3)

temperature,humidity,outlook_n,windy_n
85,85,2,0
80,90,2,1
83,86,0,0


In [45]:
#distance between 2rows
distance_between_rows(attributes.row(0),attributes.row(4))

17.748239349298849

In [46]:
example=Dataset.drop('play_n').row(4)
example

Row(temperature=68.0, humidity=80.0, outlook_n=1, windy_n=0)

In [47]:
Dataset

temperature,humidity,outlook_n,windy_n,play_n
85,85,2,0,0
80,90,2,1,0
83,86,0,0,1
70,96,1,0,1
68,80,1,0,1
65,70,1,1,0
64,65,0,1,1
72,95,2,0,0
69,70,2,0,1
75,80,1,0,1


In [49]:
closest(Dataset.exclude(4),example,5)

temperature,humidity,outlook_n,windy_n,play_n,Distance
75,80,1,0,1,7
69,70,2,0,1,10.0995
65,70,1,1,0,10.4881
72,90,0,1,1,10.8628
71,91,1,1,0,11.4455


In [48]:
def majority_class(neighbors):
    return neighbors.group('play_n').sort('count',descending=True).column('play_n').item(0)

In [51]:
def classify(training,example,k):
    nearest_neighbors=closest(training,example,k)
    return majority_class(nearest_neighbors)

In [52]:
example

Row(temperature=68.0, humidity=80.0, outlook_n=1, windy_n=0)

In [53]:
Prediction=classify(Dataset.exclude(4),example,5)

In [54]:
Prediction

1